In [41]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3.1")
llm.invoke("what is the Meaning of life")

"The meaning of life has been debated and explored by philosophers, theologians, scientists, and thinkers throughout history. There is no one definitive answer, as it can vary greatly depending on cultural, personal, and philosophical perspectives. Here are some possible interpretations:\n\n**Philosophical Perspectives:**\n\n1. **Hedonism**: Life's meaning is to pursue pleasure and happiness.\n2. **Existentialism**: Life has no inherent meaning; we must create our own purpose through individual freedom and choice.\n3. **Stoicism**: Life's meaning lies in living virtuously, cultivating inner strength, and accepting the natural order of things.\n\n**Religious Perspectives:**\n\n1. **Christianity**: The meaning of life is to know God, love Him, and live according to His will.\n2. **Buddhism**: The meaning of life is to attain enlightenment (Nirvana), free from suffering and the cycle of rebirth.\n3. **Islam**: The meaning of life is to submit to Allah's will, follow His laws, and strive f

In [42]:
ollama.pull(model="nomic-embed-text")
ollama.pull(model="llama3.1")

{'status': 'success'}

In [40]:
import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# Load the data from the web URL
url = 'https://simple.wikipedia.org/wiki/Luisa_Gonz%C3%A1lez'
loader = WebBaseLoader(url)
docs = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    system_message = (
        "You are a helpful assistant. Answer the question based on the given context. "
        "If the answer is not in the provided text, just respond with 'I do not know'."
    )
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': formatted_prompt}
    ]
    response = ollama.chat(model='llama3.1', messages=messages)
    return response['message']['content']

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    answer = ollama_llm(question, formatted_context)
    return f"Answer: {answer}\n\nUsed Text for Inference:\n{formatted_context}"

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="RAG with Llama3.1",
  description="Ask questions about the provided context",
)

# Launch the Gradio app
iface.launch()


Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.16.0, however version 4.29.0 is available, please upgrade.
--------
